In [ ]:
import requests
import pandas as pd
import notebook_utils.notebook_helpers as utils

# Service Team Routing
## Capacitated Vehicle Routing Problem with Time Windows (CVRPTW)

The ability of service providers to set service time windows allows for easier and more dependable coordination between the service provider and their customers, while increasing overall customer satisfaction.

In this scenario we have a number of service order locations with associated time windows and service times (time on-site to complete service). Each technician has an associated availability, ability to complete certain types of service, and a maximum number of service appointments per day.

### Problem Details:
- 8 Locations each with an associated demand
    - 1 Headquarters 
        - service type 1 demand: [0]
        - service type 2 demand: [1]
        - headquarters hours of operation: [5,20]
    - 7 Service Locations
        - service type 1 demand: [1, 1, 1, 0, 0, 0, 0]
        - service type 2 demand: [0, 0, 1, 1, 1, 1, 1]
        - service locations time windows: [[9,12],[9,12],[11,14],[13,16],[13,16],[13,16],[13,16]]
        - service location service times: [ 1, 1, 1.5, 0.5, 0.5, 0.5]

- 3 Delivery vehicles each with an associated capacity
    - 3 service technicians
        - capacity for service type 1: [2, 1, 0]
        - capacity for service type 2: [0, 1, 4]
        - technician availability [[9,17], [12,15], [9,17]]
        
**API Reference**: [cuOpt Server Documentation](https://docs.nvidia.com/cuopt/serv_api.html)

Below we visualize the service locations with respect to the service company headquarters. The cost from all locations to all other locations (a cost matrix) will be required for optimization. To see an example of cost matrix generation from map data or a waypoint graph, refer to the [cost_matrix_creation.ipynb](cost_matrix_creation.ipynb) notebook.  For the purpose of this simple example we will omit the cost matrix calculation.

In [ ]:
location_names       = [ "Headquarters",     "A",    "B",    "C",    "D",    "E",    "F",    "G"  ]
location_coordinates = [     [4, 4],        [1, 3], [8, 1], [2, 1], [6, 7], [0, 2], [7, 6], [5, 3] ]
location_coordinates_df = pd.DataFrame(location_coordinates, columns=['xcord', 'ycord'], index=location_names)
utils.gen_plot(location_coordinates_df).show()

### Cost Matrix

The cost matrix dictates the cost of travel between locations of interest.  The cost itself can be anything relevant to the user.  In this case we are constraining time window constraints.  When constraining time windows for locations or vehicles it is assumed (if only a single cost matrix is provided) that it represents time. 

Here is the cost(time) matrix corresponding to the locations above:

In [ ]:
time_matrix = [
 [0.00, 0.31, 0.50, 0.36, 0.36, 0.44, 0.36, 0.14],
 [0.31, 0.00, 0.72, 0.22, 0.64, 0.14, 0.67, 0.40],
 [0.50, 0.72, 0.00, 0.60, 0.63, 0.80, 0.51, 0.36],
 [0.36, 0.22, 0.60, 0.00, 0.72, 0.22, 0.70, 0.36],
 [0.36, 0.64, 0.63, 0.72, 0.00, 0.77, 0.14, 0.41],
 [0.44, 0.14, 0.80, 0.22, 0.77, 0.00, 0.80, 0.51],
 [0.36, 0.67, 0.51, 0.70, 0.14, 0.80, 0.00, 0.36],
 [0.14, 0.40, 0.36, 0.36, 0.41, 0.51, 0.36, 0.00]
]

# Create a dataframe of this matrix
time_matrix_df  = pd.DataFrame(time_matrix, 
                              index=location_coordinates_df.index, 
                              columns=location_coordinates_df.index)
time_matrix_df

### Service Locations

Setup the service location data

In [ ]:
service_location_data = {
    "location_ids": location_names,
    "service_type1_demand": [0, 1, 1, 1, 0, 0, 0, 0],
    "service_type2_demand": [0, 0, 0, 1, 1, 1, 1, 1],
    "location_earliest_time": [5, 9, 9, 11, 13, 13, 13, 13],
    "location_latest_time": [20, 12, 12, 14, 16, 16, 16,16],
    "required_service_time": [0, 1, 1, 1.5, 0.5, 0.5, 0.5, 0.5]
}
service_location_data_df = pd.DataFrame(service_location_data).set_index('location_ids')
service_location_data_df

### Vehicles

Setup vehicle/technician data

In [ ]:
n_vehicles = 3
vehicle_data = {
    "vehicle_ids": [i for i in range(n_vehicles)],
    "capacity_service_type1":[2, 1, 0],
    "capacity_service_type2":[0, 1, 4],
    "vehicle_availability_earliest":[9, 11, 9],
    "vehicle_availability_latest":[17, 15, 17]
}
vehicle_data_df = pd.DataFrame(vehicle_data).set_index('vehicle_ids')
vehicle_data_df

# Setup the cuOpt server and test its health

**NOTE**: Please update **ip** and **port** on which the server is running.

In [ ]:
ip = "0.0.0.0"
port = "5000"
url = "http://" + ip + ":" + port + "/cuopt/"

# Test health
assert requests.get(url + "health").status_code == 200

### Set Cost Matrix

In [ ]:
# set the cost matrix
data_params = {"return_data_state": False}
cost_data = {"cost_matrix": time_matrix}
response_set = requests.post(
    url + "set_cost_matrix", params=data_params, json=cost_data
)
assert response_set.status_code == 200

### Set Fleet Data

In [ ]:
# Build the vehicle data
fleet_data = {
    "vehicle_locations": [[0,0]] * n_vehicles,
    "capacities": [vehicle_data["capacity_service_type1"], vehicle_data["capacity_service_type2"]],
}

# add time windows for vehicle availability
fleet_data["vehicle_time_windows"] = list(zip(vehicle_data['vehicle_availability_earliest'],
                                              vehicle_data['vehicle_availability_latest']))

# Dispatch the vehicle data to the cuOpt server
response_set = requests.post(
    url + "set_fleet_data", json=fleet_data
)
assert response_set.status_code == 200

### Set Task Data

In [ ]:
n_locations = len(service_location_data_df)
n_vehicles = len(vehicle_data_df)

# Build the task data
task_data = {
    "task_locations": [i for i in range(n_locations)],
    # demand for service type 1 and service type 2
    "demand": [service_location_data["service_type1_demand"], service_location_data["service_type2_demand"]],
}

# add time window constraints and service time for the service locations
task_data["task_time_windows"] = list(zip(service_location_data["location_earliest_time"],
                                          service_location_data["location_latest_time"]))
task_data["service_times"] = service_location_data["required_service_time"]

# Dispatch the task data to the cuOpt server
response_set = requests.post(
    url + "set_task_data", json=task_data
)
assert response_set.status_code == 200

### Set Solver Configuration

In [ ]:
# Set the time limit 
# Set number of climbers that will try to search for an optimal routes in parallel
solver_settings = {
    "time_limit": 0.05,
    "number_of_climbers": 128,
}
# Dispatch the solver settings to the cuOpt server
response_set = requests.post(
    url + "set_solver_config", json=solver_settings
)
assert response_set.status_code == 200

### Get Optimized Routes

In [ ]:
# Solve the problem
solver_response = requests.get(url + "get_optimized_routes")

assert solver_response.status_code == 200

# Process the solver results
solver_resp = solver_response.json()["response"]["solver_response"]
solver_resp_df = utils.create_solution_dataframe(solver_resp)
if solver_resp["status"] == 0:
    print("Cost for the routing in distance: ", solver_resp["solution_cost"])
    print("Vehicle count to complete routing: ", solver_resp["num_vehicles"])
    utils.show_vehicle_routes(solver_resp_df, location_names)
else:
    print("NVIDIA cuOpt Failed to find a solution with status : ", solver_resp["status"])

**Notice** that this solution leverages the fact that vehicle 1 is the only vehicle with the ability to perform both service type 1 and service type 2.  In addition, vehicle 0 and vehicle 2 also serve the locations they are suited to service and minimize the time taken along these routes.

In [ ]:
vehicle_colors = ["red", "green", "blue"]
utils.map_vehicle_routes(location_coordinates_df, solver_resp_df, vehicle_colors).show()

In [ ]:
solver_resp_df

_____

#### SPDX-FileCopyrightText: Copyright (c) 2022 NVIDIA CORPORATION & AFFILIATES. All rights reserved.

#### SPDX-License-Identifier: MIT

Permission is hereby granted, free of charge, to any person obtaining a
copy of this software and associated documentation files (the "Software"),
to deal in the Software without restriction, including without limitation
the rights to use, copy, modify, merge, publish, distribute, sublicense,
and/or sell copies of the Software, and to permit persons to whom the
Software is furnished to do so, subject to the following conditions:
The above copyright notice and this permission notice shall be included in
all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
DEALINGS IN THE SOFTWARE.

---